In [55]:
import numpy as np
import pandas as pd

import xgboost as xgb
from sklearn.metrics import r2_score, auc

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
import pickle

import torch
import torch.utils.data as data_utils

import torch.nn as nn
import torch.nn.functional as F

import warnings
warnings.filterwarnings("ignore")

In [56]:
df = pd.read_csv('/kaggle/input/eestech-challenge-2022/Train-dataset.csv')

In [57]:
df_copy = df.copy(deep=True);

classes_num = 15; # number of output classes
values_num = len(df_copy['LITH_CODE']); # number of values in dataset
env_num = len(df_copy['DEPOSITIONAL_ENVIRONMENT'].unique());

#one_hot_class = [[0 for col in range(values_num)] for row in range(classes_num)];
one_hot_env = [[0 for col in range(values_num)] for row in range(env_num)];

help_dict_class = {
100 : 0,
200 : 1,
300 : 2,
400 : 3,
500 : 4,
600 : 5,
700 : 6,
800 : 7,
900 : 8,
1000 : 9,
1100 : 10,
1200 : 11,
1300 : 12,
1400 : 13,
1500 : 14
};

help_dict_env = {
'Continental': 0,
'Transitional': 1,
'Marine': 2,
};


for i in range(values_num):
    ind = help_dict_class[df_copy['LITH_CODE'][i]];
    df_copy['LITH_CODE'][i] = help_dict_class[df_copy['LITH_CODE'][i]];
   # one_hot_class[ind][i] = 1;

    ind = help_dict_env[df_copy['DEPOSITIONAL_ENVIRONMENT'][i]];
    one_hot_env[ind][i] = 1;
    '''
for i in range(classes_num):
    name = "classNo" + str(i);
    df_copy[name] = one_hot_class[i];
'''
for i in range(env_num):
    name = "envNo" + str(i);
    df_copy[name] = one_hot_env[i];


df_copy['X'] = (df_copy['X'] - df_copy['X'].min()) / (df_copy['X'].max() - df_copy['X'].min());
df_copy['Y'] = (df_copy['Y'] - df_copy['Y'].min()) / (df_copy['Y'].max() - df_copy['Y'].min());
df_copy['MD'] = (df_copy['MD'] - df_copy['MD'].min()) / (df_copy['MD'].max() - df_copy['MD'].min());
df_copy['GR'] = (df_copy['GR'] - df_copy['GR'].min()) / (df_copy['GR'].max() - df_copy['GR'].min());
df_copy['RT'] = (df_copy['RT'] - df_copy['RT'].min()) / (df_copy['RT'].max() - df_copy['RT'].min());
df_copy['CN'] = (df_copy['CN'] - df_copy['CN'].min()) / (df_copy['CN'].max() - df_copy['CN'].min());
df_copy['DEN'] = (df_copy['DEN'] - df_copy['DEN'].min()) / (df_copy['DEN'].max() - df_copy['DEN'].min());

df = df_copy

In [58]:
selector = 0  #0-welldropping 1-stratified 2-whole set
well_nos = [0,3,4]
y = df['LITH_CODE']
X = df[['MD','GR', 'RT', 'DEN', 'CN','envNo0', 'envNo1','envNo2']]
if selector == 0:
    wells = df['WELL'].unique()
    for well_no in well_nos:
        victim_well = wells[well_no]
        df = df[df['WELL'] != victim_well]
    y_train = df['LITH_CODE']
    X_train = df[['MD','GR', 'RT', 'DEN', 'CN','envNo0', 'envNo1','envNo2']]

    #y_test = df_well1['LITH_CODE']
    #X_test = df_well1[['MD','GR', 'RT', 'DEN', 'CN','envNo0', 'envNo1','envNo2']]
elif selector==1:
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 4, stratify = y)
else:
    X_train = X
    y_train = y
    X_test = X
    y_test = y

In [39]:
wells = df['WELL'].unique()
wells

array(['Well-4', 'Well-8', 'Well-11', 'Well-7', 'Well-10'], dtype=object)

In [5]:
from xgboost import XGBClassifier

In [59]:
model = XGBClassifier(
learning_rate =0.1,
 n_estimators=70,
 max_depth=4,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 scale_pos_weight=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


[01:13:35] WARNING: ../src/learner.cc:576: 
Parameters: { "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:13:35] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [60]:
y_pred = model.predict(X_train)
y_pred1 = [round(x) for x in y_pred]
print( round(metrics.f1_score(y_train, y_pred1, average = 'micro'), 2) )

0.85


In [61]:
def show_conf_matrix(y_test, y_pred, classes):
    
    # Calculate confusion matrix
    conf = confusion_matrix(y_test, y_pred)

    plt.figure(figsize=(12,12))
    sns.set(font_scale=1)
    sns.heatmap(conf, annot=True, annot_kws={"size": 16}, fmt="d", linewidths=.5, cmap="YlGnBu", xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted value')
    plt.ylabel('True value')
    
    plt.show()

In [ ]:
lithology_key = {100: 'Clay',
                 200: 'Siltstone/Loess',
                 300: 'Marl',
                 400: 'Clay marl',
                 500: 'Clay sandstone',
                 600: 'Sandstone',
                 700: 'Limestone',
                 800: 'Tight',
                 900: 'Dolomite',
                 1000: 'Coal',
                 1100: 'Coal clay',
                 1200: 'Marly sandstone',
                 1300: 'Sandy marl',
                 1400: 'Marl clay',
                 1500: 'Siltstone clay'
                  }

lithology_num = {0: 'Clay',
                 1: 'Siltstone/Loess',
                 2: 'Marl',
                 3: 'Clay marl',
                 4: 'Clay sandstone',
                 5: 'Sandstone',
                 6: 'Limestone',
                 7: 'Tight',
                 8: 'Dolomite',
                 9: 'Coal',
                 10: 'Coal clay',
                 11: 'Marly sandstone',
                 12: 'Sandy marl',
                 13: 'Marl clay',
                 14: 'Siltstone clay'
                  }

target_lithologys = []
labels = np.sort(y_test.unique())

for l_code in labels:
    lithology = lithology_num[l_code]
    target_lithologys.append(lithology)


print("Train set accuracy: ", round(metrics.f1_score(y_test, y_pred1, average = 'micro'), 2))

show_conf_matrix(y_pred1, y_test, target_lithologys)

print('==================================================================================')
print('Test set classification report')
print('----------------------------------------------------------------------------------')
print(classification_report(y_test, y_predict, labels=labels, target_names=target_lithologys, zero_division=0))
print('==================================================================================')

# Validate your model

In [46]:
df2 = pd.read_csv('/kaggle/input/eestech-challenge-2022/Test-dataset.csv')

In [62]:
df2_copy = df2.copy(deep = True)

env_num2 = len(df2_copy['DEPOSITIONAL_ENVIRONMENT'].unique());



for i in range(env_num2):
    name = "envNo" + str(i);
    df2_copy[name] = 0

for i in range(len(df2_copy['DEPOSITIONAL_ENVIRONMENT'])):
    if(df2_copy['DEPOSITIONAL_ENVIRONMENT'][i] == 'Continental'):
        df2_copy['envNo0'][i] = 1
    elif(df2_copy['DEPOSITIONAL_ENVIRONMENT'][i] == 'Marine'):
        df2_copy['envNo2'][i] = 1
    else:
        df2_copy['envNo1'][i] = 1

df2_copy['X'] = (df2_copy['X'] - df2_copy['X'].min()) / (df2_copy['X'].max() - df2_copy['X'].min());
df2_copy['Y'] = (df2_copy['Y'] - df2_copy['Y'].min()) / (df2_copy['Y'].max() - df2_copy['Y'].min());
df2_copy['MD'] = (df2_copy['MD'] - df2_copy['MD'].min()) / (df2_copy['MD'].max() - df2_copy['MD'].min());
df2_copy['GR'] = (df2_copy['GR'] - df2_copy['GR'].min()) / (df2_copy['GR'].max() - df2_copy['GR'].min());
df2_copy['RT'] = (df2_copy['RT'] - df2_copy['RT'].min()) / (df2_copy['RT'].max() - df2_copy['RT'].min());
df2_copy['CN'] = (df2_copy['CN'] - df2_copy['CN'].min()) / (df2_copy['CN'].max() - df2_copy['CN'].min());
df2_copy['DEN'] = (df2_copy['DEN'] - df2_copy['DEN'].min()) / (df2_copy['DEN'].max() - df2_copy['DEN'].min());

df2 = df2_copy

In [63]:
X_test2 = df2[['MD','GR', 'RT', 'DEN', 'CN','envNo0', 'envNo1','envNo2']]

In [64]:
y_pred = model.predict(X_test2)
y_pred2 = [round(x) for x in y_pred]

In [30]:
y_pred2

[4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 9,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 10,
 10,
 10,
 10,
 0,
 10,
 10,
 10,
 10,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 10,
 10,
 10,
 10,
 10,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 10,
 10,
 0,
 0,
 0,
 0,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 

In [65]:
y_pred_final = [round((x+1)*100) for x in y_pred2]

In [32]:
y_pred_final

[500,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 500,
 500,
 1000,
 500,
 500,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 500,
 500,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 100,
 100,
 100,
 100,
 100,
 100,
 1100,
 1100,
 1100,
 1100,
 100,
 1100,
 1100,
 1100,
 1100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 1100,
 1100,
 1100,
 1100,
 1100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 1100,
 1100,
 100,
 100,
 100,
 100,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 500,
 500,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 500,
 500,
 500,
 500,
 500,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,


In [66]:
df2['LITH_CODE'] = y_pred_final

In [34]:
df2

,Id,WELL,X,Y,MD,GR,RT,CN,DEN,DEPOSITIONAL_ENVIRONMENT,envNo0,envNo1,envNo2,LITH_CODE
0,1,Well-X,1.000000,0.0,0.228251,0.534758,0.036873,0.414133,0.613302,Continental,1,0,0,500
1,2,Well-X,1.000000,0.0,0.228341,0.545829,0.035148,0.424343,0.607792,Continental,1,0,0,100
2,3,Well-X,1.000000,0.0,0.228432,0.554601,0.032549,0.441150,0.602519,Continental,1,0,0,100
3,4,Well-X,1.000000,0.0,0.228523,0.561920,0.029943,0.460452,0.598898,Continental,1,0,0,100
4,5,Well-X,1.000000,0.0,0.228613,0.569707,0.029649,0.479266,0.599134,Continental,1,0,0,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28992,28993,Well-Z,0.375895,1.0,0.972546,0.683857,0.017824,0.333589,0.769776,Marine,0,0,1,1100
28993,28994,Well-Z,0.375895,1.0,0.972637,0.677461,0.018052,0.330640,0.772216,Marine,0,0,1,1100
28994,28995,Well-Z,0.375895,1.0,0.972728,0.677265,0.018346,0.325788,0.774183,Marine,0,0,1,1100
28995,28996,Well-Z,0.375895,1.0,0.972819,0.683176,0.018785,0.319837,0.776151,Marine,0,0,1,1100


In [67]:
submission = df2[['Id','LITH_CODE']]

In [68]:
submission

,Id,LITH_CODE
0,1,500
1,2,100
2,3,100
3,4,100
4,5,100
...,...,...
28992,28993,1100
28993,28994,1100
28994,28995,1100
28995,28996,1100


In [69]:
submission.to_csv('submission4.csv', index=False)